# Microsoft Power BI Cmdlets for Windows PowerShell and PowerShell Core
The following cmdlets are available on PowerShell Gallery and can be installed in an elevated PowerShell session:
- [Microsoft Power BI](https://docs.microsoft.com/en-us/powershell/power-bi/overview?view=powerbi-ps)
- [Data Gateway](https://docs.microsoft.com/en-us/powershell/module/datagateway/?view=datagateway-ps)

In [ ]:
$modules = @("MicrosoftPowerBIMGMT", "DataGateway")

foreach ( $m in $modules ) 
{
    if (Get-Module -ListAvailable -Name $m) {
        write-host "Module $m is already imported."
    } 
    else {
        Install-Module -Name $m -Force -Scope CurrentUser
        Import-Module $m
        write-host "Module $m is now imported."
    }
}

# Available Cmdlets
Returns the available cmdlets from either the MicrosoftPowerBIMGMT or DataGateway module.

In [ ]:
Get-Help *PowerBI* | Sort-Object Name | Format-Table -AutoSize

In [ ]:
Get-Help *DataGateway* | Sort-Object Name | Format-Table -AutoSize

# Connect to Power BI service
Login with Azure Active Directory user authentication or Service Principal credentials.

For Service Principal:
- Connect-PowerBIServiceAccount -ServicePrincipal -Credential (Get-Credential)

In [2]:
Connect-PowerBIServiceAccount


Environment : Public
TenantId    : 72f988bf-86f1-41af-91ab-2d7cd011db47
UserName    : alpowers@microsoft.com




# Scope

Indicates the scope of the call.
- -Scope Individual
    - Returns only workspaces assigned to the caller.
- -Scope Organization
    - Returns all workspaces within a tenant (must be an administrator to initiate).

In [3]:
$scopeLevel = "Individual"

# Return a list of Power BI workspaces

Return a listing of the Power BI workspaces including Id, Name, IsReadOnly and IsOnDedicatedCapacity.

To list all workspaces, use the parameter -All

In [ ]:
$pbiWorkspaces = Get-PowerBIWorkspace -Scope $scopeLevel -All
Write-Host "Total Number of Workspaces: $($pbiWorkspaces.Count)`n"

Get-PowerBIWorkspace -Scope Individual -First 5

# Return a list of Power BI datasets in a workspace

Return a listing of the Power BI datasets within a workspace including the additional attributes WorkspaceId and Type.

In [ ]:
$pbiWorkspaces = Get-PowerBIWorkspace -Scope $scopeLevel -First 5

$pbiDatasets = @()

ForEach ($workspace in $pbiWorkspaces) {

    $pbiDatasets += Get-PowerBIDataset -Scope $scopeLevel -WorkspaceId $workspace.Id | select *, @{Name="WorkspaceId"; Expression={ $workspace.Id } }, @{Name="Type"; Expression={ "Dataset" } }
    
}

$pbiDatasets | Sort-Object -Property Id -Unique

# Return a list of Power BI dataflows in a workspace

Return a listing of the Power BI dataflows within a workspace including the additional attributes WorkspaceId and Type.

In [ ]:
$pbiWorkspaces = Get-PowerBIWorkspace -Scope $scopeLevel -First 5

$pbiDataflows = @()

ForEach ($workspace in $pbiWorkspaces) {

    $pbiDataflows += Get-PowerBIDataflow -Scope $scopeLevel -WorkspaceId $workspace.Id | select *, @{Name="WorkspaceId"; Expression={ $workspace.Id } }, @{Name="Type"; Expression={ "Dataflow" } }
    
}

$pbiDataflows | Sort-Object -Property Id -Unique

# Return a list of Power BI datasources in a dataflow in a workspace

Return a listing of the Power BI datasources in a dataflow within a workspace including the additional attributes WorkspaceId, DataflowId and Type.

In [ ]:
$pbiWorkspaces = Get-PowerBIWorkspace -Scope $scopeLevel -First 5

$pbiDataflowDatasources = @()

ForEach ($workspace in $pbiWorkspaces) {

    $pbiDataflows += Get-PowerBIDataflow -Scope $scopeLevel -WorkspaceId $workspace.Id

    ForEach ($dataflow in $pbiDataflows) {

        $pbiDataflowDatasources += Get-PowerBIDataflowDatasource -Scope $scopeLevel -WorkspaceId $workspace.Id -DataflowId $dataflow.Id | select *, @{Name="WorkspaceId"; Expression={ $workspace.Id } }, @{Name="DataflowId"; Expression={ $dataflow.Id } }, @{Name="Type"; Expression={ "Dataflow Datasource" } }

    }
    
}

$pbiDataflowDatasources | Sort-Object -Property Id -Unique

# Return a list of Power BI reports in a workspace

Return a listing of the Power BI reports within a workspace including the additional attributes WorkspaceId and Type.

In [ ]:
$pbiWorkspaces = Get-PowerBIWorkspace -Scope $scopeLevel -First 5

$pbiReports = @()

ForEach ($workspace in $pbiWorkspaces) {

    $pbiReports += Get-PowerBIReport -Scope $scopeLevel -WorkspaceId $workspace.Id | select *, @{Name="WorkspaceId"; Expression={ $workspace.Id } }, @{Name="Type"; Expression={ "Report" } }
    
}

$pbiReports | Sort-Object -Property Id -Unique

# Return a list of Power BI dashboards in a workspace

Return a listing of the Power BI dasbhoards within a workspace including the additional attributes WorkspaceId and Type.

In [ ]:
$pbiWorkspaces = Get-PowerBIWorkspace -Scope $scopeLevel -First 5

$pbiDashboards = @()

ForEach ($workspace in $pbiWorkspaces) {

    $pbiDashboards += Get-PowerBIDashboard -Scope $scopeLevel -WorkspaceId $workspace.Id | select *, @{Name="WorkspaceId"; Expression={ $workspace.Id } }, @{Name="Type"; Expression={ "Dashboard" } }
    
}

$pbiDashboards | Sort-Object -Property Id -Unique

# Return a list of Power BI tiles on a dashboard in a workspace

Return a listing of the Power BI tiles on a dashboard within a workspace including the additional attributes WorkspaceId, DashboardId and Type.



In [ ]:
$pbiWorkspaces = Get-PowerBIWorkspace -Scope $scopeLevel -First 5

$pbiTiles = @()

ForEach ($workspace in $pbiWorkspaces) {

    $pbiDashboards = Get-PowerBIDashboard -Scope $scopeLevel -WorkspaceId $workspace.Id

    ForEach ($dashboard in $pbiDashboards) {

        $pbiTiles += Get-PowerBITile -Scope $scopeLevel -WorkspaceId $workspace.Id -DashboardId $dashboard.Id | select *, @{Name="WorkspaceId"; Expression={ $workspace.Id } }, @{Name="DashboardId"; Expression={ $dashboard.Id } }, @{Name="Type"; Expression={ "Tile" } }

    }
    
}

$pbiTiles | Sort-Object -Property Id -Unique

# Return a list of the Power BI import state of a dataset in a workspace

Return a listing of the Power BI import state in a workspace including the additional attribute WorkspaceId.

- ImportState
    - Succeeded
    - Failed

In [4]:
$pbiWorkspaces = Get-PowerBIWorkspace -Scope $scopeLevel -First 5

$pbiImports = @()

ForEach ($workspace in $pbiWorkspaces) {

    $pbiImports += Get-PowerBIImport -Scope $scopeLevel -WorkspaceId $workspace.Id | select *, @{Name="WorkspaceId"; Expression={ $workspace.Id } }
    
}

$pbiImports | Sort-Object -Property Id -Unique


Id          : 27785472-0ad9-4d66-b500-2a10c941e66a
Name        : beta-SCR FY19 IPCS Report 2.pbix
ImportState : Succeeded
Reports     : {beta-SCR FY19 IPCS Report 2}
Datasets    : {beta-SCR FY19 IPCS Report 2}
Created     : 3/5/2019 10:34:34 PM
Updated     : 6/21/2019 2:56:31 PM
WorkspaceId : e9d06a02-42f6-4471-8f0f-738311fe6adb

Id          : 5e041fee-de85-4766-a629-e99303d4bc9d
Name        : beta-SCR FY19 IPCS Report.pbix
ImportState : Failed
Reports     : {beta-SCR FY19 IPCS Report}
Datasets    : {beta-SCR FY19 IPCS Report}
Created     : 2/15/2019 3:49:21 PM
Updated     : 3/5/2019 5:18:35 PM
WorkspaceId : e9d06a02-42f6-4471-8f0f-738311fe6adb

Id          : 7969059d-f74b-4a90-b6a6-861eec456ee9
Name        : US Sales Analysis.pbix
ImportState : Succeeded
Reports     : {}
Datasets    : {US Sales Analysis}
Created     : 5/11/2017 5:08:57 PM
Updated     : 5/11/2017 5:08:57 PM
WorkspaceId : 481b3fc0-4863-4720-a316-e7b35d2c1a62

Id          : 95d18dab-3ab2-453f-aedf-ad9ae5ff4d46
Name     

# Return a list of the Power BI data sources of a dataset in a workspace

Return a listing of the Power BI data sources of a dataset in a workspace including the additional attributes WorkspaceId and Type.

In [ ]:
$pbiWorkspaces = Get-PowerBIWorkspace -Scope $scopeLevel -First 5

$pbiDatasets = @()

ForEach ($workspace in $pbiWorkspaces) {

    $pbiDashboards = Get-PowerBIDashboard -Scope $scopeLevel -WorkspaceId $workspace.Id

    ForEach ($dashboard in $pbiDashboards) {

        $pbiTiles += Get-PowerBITile -Scope $scopeLevel -WorkspaceId $workspace.Id -DashboardId $dashboard.Id | select *, @{Name="WorkspaceId"; Expression={ $workspace.Id } }, @{Name="DashboardId"; Expression={ $dashboard.Id } }, @{Name="Type"; Expression={ "Tile" } }

    }
    
}

$pbiTiles | Sort-Object -Property Id -Unique